<a href="https://colab.research.google.com/github/JulianMurillo413/JulianAndresMurilloGarzon-NRC-60-69131/blob/main/ProyectoExpoPer%C3%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import csv
import pandas as pd

# Clase Nodo: representa cada elemento de la lista doblemente enlazada
class Node:
    def __init__(self, data):
        self.data = data  # Contenido del nodo (un diccionario de datos)
        self.next = None  # Referencia al siguiente nodo
        self.prev = None  # Referencia al nodo anterior

# Clase DoublyLinkedList: implementa la estructura de lista doblemente enlazada
class DoublyLinkedList:
    def __init__(self):
        self.head = None  # Primer nodo de la lista
        self.tail = None  # Último nodo de la lista

    # Inserta un nodo con los datos especificados al final de la lista
    def insert_at_end(self, data):
        new_node = Node(data)
        if self.head is None:  # Si la lista está vacía
            self.head = self.tail = new_node
        else:  # Agregar al final de la lista
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node

    # Obtiene todos los datos de la lista en formato de lista de diccionarios
    def get_data(self):
        data = []
        current = self.head
        while current:  # Recorre todos los nodos
            data.append(current.data)
            current = current.next
        return data

    # Busca un término en los datos de la lista (insensible a mayúsculas)
    def search(self, term):
        results = []
        current = self.head
        while current:
            # Busca si el término aparece en algún valor del nodo actual
            if any(term.lower() in str(value).lower() for value in current.data.values()):
                results.append(current.data)
            current = current.next
        return results

    # Ordena los nodos de la lista según una columna específica
    def sort_by_column(self, column_name, descending=False):
        # Obtiene los datos actuales y los ordena
        data = self.get_data()
        data.sort(key=lambda x: x.get(column_name, ""), reverse=descending)

        # Reconstruye la lista con los datos ordenados
        self.head = self.tail = None
        for item in data:
            self.insert_at_end(item)

# Función para cargar datos desde un archivo (puede ser .txt, .csv o .xlsx)
def load_data(file_path):
    data = []
    if file_path.endswith('.txt') or file_path.endswith('.csv'):  # Archivos de texto/CSV
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                data.append(row)
    elif file_path.endswith('.xlsx'):  # Archivos de Excel
        df = pd.read_excel(file_path)
        data = df.to_dict(orient='records')  # Convierte los datos a una lista de diccionarios
    else:
        raise ValueError("Formato de archivo no soportado. Usa .txt, .csv o .xlsx.")
    return data

# Clase principal de la aplicación: controla la interfaz gráfica
class Application:
    def __init__(self, root):
        self.root = root
        self.root.title("Lista doblemente enlazada - GUI")
        self.root.geometry("800x700")
        self.root.configure(bg="#2E2E2E")  # Fondo gris oscuro

        self.list = DoublyLinkedList()  # Instancia de la lista doblemente enlazada
        self.columns = []  # Columnas detectadas en los datos

        # Frame para cargar archivos
        self.load_frame = tk.Frame(root, bg="#FF5733")
        self.load_frame.pack(pady=10, fill=tk.X)

        # Botón para cargar un archivo
        self.load_button = tk.Button(self.load_frame, text="Cargar archivo", bg="#FFC300", fg="black", command=self.load_file)
        self.load_button.pack(side=tk.LEFT, padx=5)

        # Botón para mostrar datos
        self.show_button = tk.Button(self.load_frame, text="Mostrar datos", bg="#DAF7A6", fg="black", command=self.show_data)
        self.show_button.pack(side=tk.LEFT, padx=5)

        # Tabla para mostrar los datos cargados
        self.treeview = ttk.Treeview(root, show="headings")
        self.treeview.pack(pady=10, fill=tk.BOTH, expand=True)

        # Frame para opciones de ordenamiento
        self.options_frame = tk.Frame(root, bg="#581845")
        self.options_frame.pack(pady=10, fill=tk.X)

        # Botón para ordenar por código (ascendente)
        self.sort_by_code_button = tk.Button(self.options_frame, text="Ordenar por código (Asc)", bg="#FFC300", fg="black", command=self.sort_by_code)
        self.sort_by_code_button.pack(side=tk.LEFT, padx=5)

        # Botón para ordenar por nombre (descendente)
        self.sort_by_name_button = tk.Button(self.options_frame, text="Ordenar por nombre (Desc)", bg="#FFC300", fg="black", command=self.sort_by_name)
        self.sort_by_name_button.pack(side=tk.LEFT, padx=5)

        # Frame para la búsqueda de datos
        self.search_frame = tk.Frame(root, bg="#1C2833")
        self.search_frame.pack(pady=10, fill=tk.X)

        # Etiqueta y campo de entrada para buscar
        self.search_label = tk.Label(self.search_frame, text="Buscar:", bg="#1C2833", fg="white")
        self.search_label.pack(side=tk.LEFT, padx=5)
        self.search_entry = tk.Entry(self.search_frame, width=30)
        self.search_entry.pack(side=tk.LEFT, padx=5)

        # Botón para realizar la búsqueda
        self.search_button = tk.Button(self.search_frame, text="Buscar", bg="#FFC300", fg="black", command=self.search_data)
        self.search_button.pack(side=tk.LEFT, padx=5)

    # Carga un archivo seleccionado y llena la lista con los datos
    def load_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Archivos de datos", "*.txt *.csv *.xlsx")])
        if file_path:
            try:
                data = load_data(file_path)
                if not data:
                    raise ValueError("El archivo está vacío o no contiene datos válidos.")
                self.list = DoublyLinkedList()  # Reinicia la lista
                self.columns = list(data[0].keys())  # Obtiene las columnas de los datos
                for item in data:
                    self.list.insert_at_end(item)  # Agrega cada fila a la lista
                self.setup_treeview()
                messagebox.showinfo("Éxito", f"Archivo cargado con {len(data)} registros.")
                self.show_data()
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo cargar el archivo: {e}")

    # Configura las columnas del Treeview dinámicamente según los datos
    def setup_treeview(self):
        self.treeview.delete(*self.treeview.get_children())  # Limpia los datos anteriores
        self.treeview["columns"] = self.columns
        for col in self.columns:
            self.treeview.heading(col, text=col)
            self.treeview.column(col, width=100)

    # Muestra los datos de la lista en el Treeview
    def show_data(self):
        self.treeview.delete(*self.treeview.get_children())  # Limpia la tabla
        current = self.list.head
        if not current:
            messagebox.showinfo("Info", "No hay datos para mostrar.")
            return
        while current:
            values = [current.data.get(col, "") for col in self.columns]
            self.treeview.insert("", tk.END, values=values)
            current = current.next

    # Ordena los datos por la columna "Código"
    def sort_by_code(self):
        self.list.sort_by_column("Código")
        self.show_data()

    # Ordena los datos por la columna "Nombre" en orden descendente
    def sort_by_name(self):
        self.list.sort_by_column("Nombre", descending=True)
        self.show_data()

    # Busca un término en los datos y muestra los resultados en el Treeview
    def search_data(self):
        term = self.search_entry.get().strip()
        if not term:
            messagebox.showwarning("Advertencia", "Introduce un término para buscar.")
            return
        results = self.list.search(term)
        self.treeview.delete(*self.treeview.get_children())
        for item in results:
            values = [item.get(col, "") for col in self.columns]
            self.treeview.insert("", tk.END, values=values)

# Punto de entrada principal de la aplicación
if __name__ == "__main__":
    root = tk.Tk()
    app = Application(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable